In [ ]:
# pip install autogluon.tabular
# pip install pandas

import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import os
# Установка сида для воспроизводимости (опционально)
# os.environ["PYTHONHASHSEED"] = "42"

tab

In [ ]:
# --- 1. Загрузка данных ---
# Предположим, файлы лежат в папке 'data'
train_path = 'data/train.csv'
test_path = 'data/test.csv'
submission_path = 'data/sample_submission.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_submission = pd.read_csv(submission_path)

print(f"Тренировочный датасет: {train_data.shape}")
print(f"Тестовый датасет: {test_data.shape}")
print(f"Целевая переменная (первые 5): {train_data['target'].head()}")

# --- 2. Подготовка данных ---
# Убедитесь, что в тренировочных данных нет строк из теста (убедитесь в отсутствии утечки данных)
# В соревнованиях часто присутствуют id-столбцы, которые нужно удалить или использовать правильно
# Оставим все столбцы, кроме id и target, для тренировки. AutoGluon сам определит типы.

label = 'target' # Имя целевой переменной
# Опционально: удалить id-столбец из тренировочных данных, если он не нужен для обучения
# train_data = train_data.drop(columns=['id'])

# --- 3. Настройка и обучение AutoGluon ---
# Путь для сохранения моделей
model_output_path = 'ag_models_tabular'

# Создание и обучение предиктора
predictor = TabularPredictor(
    label=label,
    path=model_output_path
)

# Обучение с настройками для соревнования
predictor.fit(
    train_data=train_data,
    presets='best_quality',  # Максимизирует качество, использует стекинг и бэггинг
    time_limit=3600,  # Обучение не более 1 часа (60*60 секунд)
    num_bag_folds=5,  # Включить бэггинг для стабильности (5 фолдов)
    num_stack_levels=2, # Включить стекинг (2 уровня)
    # eval_metric='roc_auc' # Укажите метрику, если отличается от стандартной (например, для бинарной классификации)
)

# --- 4. Анализ обученных моделей ---
print("\n--- Leaderboard обученных моделей ---")
leaderboard = predictor.leaderboard(silent=False)
print(leaderboard)

print("\n--- Лучшая модель ---")
best_model = predictor.get_model_best()
print(f"Лучшая модель: {best_model}")

# --- 5. Получение предсказаний на тестовом датасете ---
# Для бинарной классификации:
# predictor.predict() возвращает предсказанный класс (0 или 1)
# predictor.predict_proba() возвращает вероятности
test_predictions_proba = predictor.predict_proba(test_data)
# Для бинарной задачи, обычно нас интересует вероятность положительного класса
final_predictions = test_predictions_proba.iloc[:, 1] # или test_predictions_proba.loc[:, 1]

# Если нужен класс, а не вероятность:
# final_predictions = predictor.predict(test_data)

# --- 6. Формирование и сохранение сабмита ---
# Обычно соревнования требуют файл с 'id' и 'target' (или другое имя)
submission = sample_submission.copy()
# Пример: если в sample_submission есть столбец 'id', он уже там
# Заменяем столбец 'target' (или как он там называется) на наши предсказания
submission.iloc[:, 1] = final_predictions # Заменить столбец 1 (обычно target) на предсказания
# submission['target'] = final_predictions # Альтернативный способ, если вы знаете имя столбца

submission_path_final = 'submission_autogluon_tabular.csv'
submission.to_csv(submission_path_final, index=False)
print(f"\nСабмит сохранен в: {submission_path_final}")


text

In [ ]:
# --- 1. Загрузка данных ---
train_text_path = 'data/train_text.csv'
test_text_path = 'data/test_text.csv'

train_text_data = pd.read_csv(train_text_path)
test_text_data = pd.read_csv(test_text_path)

print(f"Тренировочный датасет: {train_text_data.shape}")
print(f"Тестовый датасет: {test_text_data.shape}")
print(f"Пример текста (первые 2): \n{train_text_data['text'].head(2)}")

# --- 2. Подготовка данных ---
label = 'sentiment' # Имя целевой переменной
text_column = 'text' # Имя столбца с текстом

# AutoGluon автоматически распознает текстовые столбцы и использует соответствующие модели (например, Transformer)
# Важно: убедитесь, что текст в столбце 'text' не содержит NaN или других неожиданных значений
# train_text_data = train_text_data.dropna(subset=[text_column, label]) # Опционально очистить

# --- 3. Настройка и обучение AutoGluon ---
model_output_path_text = 'ag_models_text'

predictor_text = TabularPredictor(
    label=label,
    path=model_output_path_text
)

predictor_text.fit(
    train_data=train_text_data,
    presets='best_quality',
    time_limit=7200,  # 2 часа для текста может быть разумным
    num_bag_folds=5,
    num_stack_levels=1, # Текстовые модели могут быть тяжелыми, уровень 1 - компромисс
    # AutoGluon использует модели, такие как TextPredictor под капотом для текста
)

# --- 4. Анализ ---
print("\n--- Leaderboard (Текстовые данные) ---")
leaderboard_text = predictor_text.leaderboard(silent=False)
print(leaderboard_text)

# --- 5. Получение предсказаний ---
test_predictions_text = predictor_text.predict(test_text_data)
# Или, если нужна вероятность (для мультикласса)
# test_predictions_text_proba = predictor_text.predict_proba(test_text_data)

# --- 6. Формирование и сохранение сабмита ---
# Предположим, сабмит должен содержать id и sentiment
submission_text = pd.DataFrame({
    'id': test_text_data['id'], # Предполагаем, что id есть в тестовом датасете
    'sentiment': test_predictions_text
})

submission_path_text_final = 'submission_autogluon_text.csv'
submission_text.to_csv(submission_path_text_final, index=False)
print(f"\nСабмит текста сохранен в: {submission_path_text_final}")

image

In [ ]:
# pip install autogluon.multimodal
from autogluon.multimodal import MultiModalPredictor

# --- 1. Загрузка данных ---
train_img_path = 'data/train_images.csv'
test_img_path = 'data/test_images.csv'

train_img_data = pd.read_csv(train_img_path)
test_img_data = pd.read_csv(test_img_path)

# Пути к изображениям должны быть относительными или абсолютными
# Пример содержимого train_img_data: ['img_path', 'label']
# Пример: ['train_images/001.jpg', 'cat']
print(f"Тренировочный датасет: {train_img_data.shape}")
print(f"Пример пути к изображению: {train_img_data['img_path'].iloc[0]}")

# --- 2. Подготовка данных ---
label_img = 'label' # Имя целевой переменной
image_column = 'img_path' # Имя столбца с путем к изображению

# Убедитесь, что пути к изображениям корректны и изображения существуют
# os.path.exists(train_img_data[image_column].iloc[0])

# --- 3. Настройка и обучение AutoGluon (MultiModal) ---
model_output_path_img = 'ag_models_image'

predictor_img = MultiModalPredictor(
    label=label_img,
    path=model_output_path_img
)

predictor_img.fit(
    train_data=train_img_data,
    presets='best_quality', # Или 'medium_quality' для скорости
    time_limit=10800, # 3 часа для изображений
    # num_bag_folds=5, # MultiModalPredictor не всегда поддерживает bagging так же, как Tabular
    # num_stack_levels=1, # См. выше
)

# --- 4. Анализ ---
print("\n--- Leaderboard (Изображения) ---")
# MultiModalPredictor может не иметь leaderboard в том же формате
# predictor_img.leaderboard(silent=False) # Проверить, поддерживается ли

# --- 5. Получение предсказаний ---
test_predictions_img = predictor_img.predict(test_img_data)

# --- 6. Формирование и сохранение сабмита ---
submission_img = pd.DataFrame({
    'id': test_img_data['id'], # Предполагаем, что id есть
    'label': test_predictions_img
})

submission_path_img_final = 'submission_autogluon_image.csv'
submission_img.to_csv(submission_path_img_final, index=False)
print(f"\nСабмит изображений сохранен в: {submission_path_img_final}")

audio

In [ ]:
# pip install autogluon.multimodal torchaudio librosa

# --- 1. Загрузка данных ---
train_audio_path = 'data/train_audio.csv'
test_audio_path = 'data/test_audio.csv'

train_audio_data = pd.read_csv(train_audio_path)
test_audio_data = pd.read_csv(test_audio_path)

# Пример содержимого train_audio_: ['audio_path', 'sound_type']
# Пример: ['train_audio/001.wav', 'dog_bark']
print(f"Тренировочный датасет: {train_audio_data.shape}")
print(f"Пример пути к аудио: {train_audio_data['audio_path'].iloc[0]}")

label_audio = 'sound_type'
audio_column = 'audio_path' # Имя столбца с путем к аудиофайлу

# AutoGluon Multimodal может обрабатывать аудио через MultiModalPredictor
# Важно: аудио-фичи могут быть сложными для обработки. MultiModalPredictor использует предобученные модели.

# --- 2. Подготовка данных ---
# Убедитесь, что пути к аудио корректны
# os.path.exists(train_audio_data[audio_column].iloc[0])

# AutoGluon ожидает, что столбец audio_column содержит пути к файлам
# Он сам загрузит и обработает аудио

# --- 3. Обучение ---
model_output_path_audio = 'ag_models_audio'
predictor_audio = MultiModalPredictor(label=label_audio, path=model_output_path_audio)

predictor_audio.fit(
    train_data=train_audio_data,
    presets='best_quality',
    time_limit=10800, # 3 часа для аудио может быть разумным
    # hyperparameters={'model.hf_text.checkpoint_name': 'microsoft/wavlm-base'} # Пример кастомизации
)

# --- 4. Анализ ---
print("\n--- Leaderboard (Аудио) ---")
# predictor_audio.leaderboard(silent=False) # Проверить, поддерживается ли

# --- 5. Получение предсказаний ---
test_predictions_audio = predictor_audio.predict(test_audio_data)

# --- 6. Формирование и сохранение сабмита ---
submission_audio = pd.DataFrame({
    'id': test_audio_data['id'], # Предполагаем, что id есть
    'sound_type': test_predictions_audio
})

submission_path_audio_final = 'submission_autogluon_audio.csv'
submission_audio.to_csv(submission_path_audio_final, index=False)
print(f"\nСабмит аудио сохранен в: {submission_path_audio_final}")

ts

In [ ]:
# pip install autogluon.timeseries

# --- 1. Загрузка данных ---
# Временные ряды должны быть в формате с колонками: item_id, timestamp, target
# Пример: item_id, timestamp, value
#         1,       2023-01-01, 100
#         1,       2023-01-02, 105
#         2,       2023-01-01, 200

ts_train_path = 'data/train_ts.csv'
ts_test_path = 'data/test_ts.csv'

ts_train_data = pd.read_csv(ts_train_path)
# ts_test_data = pd.read_csv(ts_test_path) # Часто не нужен для TimeSeriesPredictor

# Пример: предсказать 10 шагов вперед
prediction_length = 10

# --- 2. Подготовка данных ---
# Убедитесь, что timestamp в формате datetime
# ts_train_data['timestamp'] = pd.to_datetime(ts_train_data['timestamp'])

# --- 3. Обучение ---
model_output_path_ts = 'ag_models_timeseries'
predictor_ts = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path=model_output_path_ts,
    target='value', # Имя столбца с целевым значением
    freq='D', # Частота (например, D - daily, H - hourly). Опционально.
)

predictor_ts.fit(
    train_data=ts_train_data,
    presets='best_quality',
    time_limit=7200,
)

# --- 4. Анализ ---
print("\n--- Leaderboard (Ряды Времени) ---")
leaderboard_ts = predictor_ts.leaderboard(ts_train_data)
print(leaderboard_ts)

# --- 5. Получение предсказаний ---
# Прогноз для всех item_id в ts_train_data
forecast_data = predictor_ts.predict(ts_train_data)

# --- 6. Формирование и сохранение сабмита ---
# Результат forecast_data содержит колонки: item_id, timestamp, forecast (и др.)
# Соревнование может требовать специфический формат.
# Пример: агрегировать прогнозы (например, среднее по прогнозу) для каждого item_id
# или сохранить все прогнозы по времени.

# Агрегируем по item_id (например, среднее значение прогноза)
# forecast_summary = forecast_data.groupby('item_id')['mean'].mean().reset_index()

# Или оставляем как есть, если соревнование принимает такой формат
submission_ts = forecast_data.reset_index(drop=True) # Сброс индекса для чистоты

submission_path_ts_final = 'submission_autogluon_timeseries.csv'
submission_ts.to_csv(submission_path_ts_final, index=False)
print(f"\nСабмит временных рядов сохранен в: {submission_path_ts_final}")

img+tab

In [ ]:
# pip install autogluon.multimodal

# --- 1. Загрузка данных ---
train_multi_path = 'data/train_multi.csv'
test_multi_path = 'data/test_multi.csv'

train_multi_data = pd.read_csv(train_multi_path)
test_multi_data = pd.read_csv(test_multi_path)

# Пример содержимого train_multi_: ['img_path', 'bedrooms', 'bathrooms', 'sqft', 'price']
# Пример: ['train_images/house_001.jpg', 3, 2, 1500, 300000]
print(f"Тренировочный датасет: {train_multi_data.shape}")
print(f"Пример данных: \n{train_multi_data.head(2)}")

label_multi = 'price'
image_column_multi = 'img_path'
# Остальные столбцы (bedrooms, bathrooms, sqft) будут обработаны как табличные признаки

# --- 2. Подготовка данных ---
# Убедитесь, что пути к изображениям корректны
# os.path.exists(train_multi_data[image_column_multi].iloc[0])

# --- 3. Обучение ---
model_output_path_multi = 'ag_models_multi'
predictor_multi = MultiModalPredictor(label=label_multi, path=model_output_path_multi)

predictor_multi.fit(
    train_data=train_multi_data,
    presets='best_quality',
    time_limit=14400, # 4 часа для мультимодальных данных может быть разумным
    # hyperparameters={'model.names': ['timm_image', 'hf_text', 'NumericalTransformer']} # Пример кастомизации
)

# --- 4. Анализ ---
print("\n--- Leaderboard (Мультимодальные данные) ---")
# predictor_multi.leaderboard(silent=False) # Проверить, поддерживается ли

# --- 5. Получение предсказаний ---
test_predictions_multi = predictor_multi.predict(test_multi_data)

# --- 6. Формирование и сохранение сабмита ---
submission_multi = pd.DataFrame({
    'id': test_multi_data['id'], # Предполагаем, что id есть
    'price': test_predictions_multi
})

submission_path_multi_final = 'submission_autogluon_multimodal.csv'
submission_multi.to_csv(submission_path_multi_final, index=False)
print(f"\nСабмит мультимодальных данных сохранен в: {submission_path_multi_final}")